In [8]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [9]:
from tqdm.notebook import tqdm
import librosa
import whisper
import os
model = whisper.load_model("base")

In [10]:
model = model.cuda()

In [11]:
from PIL import Image
image = Image.open("cat.jpg")
inputs = processor(
    text=["a photo of a cat", "a photo of a dog"],
    images=image,
    return_tensors="pt",
    padding=True,
)

In [12]:
from video_processor import VideoProcessor

In [13]:
vp = VideoProcessor(
  clip_model.text_model.cuda(),
  clip_model.vision_model.cuda(),
  processor.tokenizer,
  processor.image_processor  
) 

In [7]:
images = vp.image_processor(
    [image,image]
)['pixel_values']

In [8]:
images[0].shape

(3, 224, 224)

In [9]:
import torch
torch.tensor(
    images
).shape

/tmp/ipykernel_1712591/1127037752.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  torch.tensor(


torch.Size([2, 3, 224, 224])

In [7]:
out = vp.extract_frames_and_audio("/DATA/sarmistha_2221cs21/basha/VideoMAE/dataset/_A5qfpLTbns.mp4")

/DATA/sarmistha_2221cs21/basha/VideoMAE/Design/video_processor.py:97: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, _ = librosa.load(video_path, sr=16000)
/DATA/sarmistha_2221cs21/anaconda3/envs/basha/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
100%|██████████| 180/180 [02:36<00:00,  1.15it/s]
/DATA/sarmistha_2221cs21/basha/VideoMAE/Design/video_processor.py:122: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  converted_clips = torch.tensor(frames)


In [14]:
import pickle
with open("data.pkl",'rb') as f:
    out = pickle.load(f)

In [15]:
vision = vp.vision_encoder(out[1])

Encoding frames


ValueError: Invalid image shape. Expected either 4 or 3 dimensions, but got 1 dimensions.

In [17]:
out = vp.image_model(**processor.image_processor([image,image], return_tensors="pt").to("cuda"))

In [23]:
out.pooler_output.mean(axis=0).shape

torch.Size([768])

In [16]:
processor.tokenizer('basha')

{'input_ids': [49406, 932, 3337, 49407], 'attention_mask': [1, 1, 1, 1]}

In [19]:

out = clip_model.text_model(
    input_ids = inputs['input_ids'],
    attention_mask = inputs['attention_mask']
)

In [32]:
out.last_hidden_state.shape

torch.Size([2, 7, 512])

In [30]:
out.pooler_output

tensor([[-0.5152,  0.1658,  0.8876,  ..., -0.0675, -0.4551, -1.7960],
        [ 0.0426,  0.0189,  1.2740,  ..., -0.4217, -0.4393, -1.3016]],
       grad_fn=<IndexBackward0>)

In [29]:
out.last_hidden_state[0]

tensor([ 6.4560e-01, -5.7594e-01,  5.0856e-01,  1.7359e+00,  7.4192e-01,
         9.6155e-01,  7.5606e-02,  5.1116e-01,  7.3742e-02,  2.1503e-01,
         7.6160e-01, -1.4468e-01, -7.0118e-02,  1.0690e-01, -3.0795e-01,
        -7.1974e-01,  5.3679e-01, -2.2584e-02, -4.2953e-01,  3.6920e-01,
         3.8226e-01, -3.3620e-02, -1.6207e-01,  4.9019e-01, -7.5640e-02,
        -9.1454e-03, -4.4394e-02, -2.5241e-01, -6.0639e-01,  1.0829e+00,
         1.5226e-01,  3.7181e-02, -1.0843e+00, -3.0456e-01, -6.2940e-01,
         3.8670e-02, -2.7472e-01, -6.9654e-01,  2.9603e-01,  2.3200e-01,
         8.5503e-02,  2.6541e-01,  4.9576e-02,  4.4805e-01, -1.7676e-01,
         7.1217e-01,  2.7446e-01, -3.2174e-01,  5.9483e-01,  9.7697e-02,
         4.0697e-01,  5.5864e-01,  2.6820e+00,  5.6556e-01,  4.6075e-01,
        -3.7861e-01,  4.9899e-01,  3.8797e-01,  1.6534e-01, -4.5438e-01,
         4.3459e-01,  3.3482e-01, -6.9586e-02,  3.5913e-01, -6.5767e-01,
         2.6411e-01,  9.3526e-01,  3.4101e-01, -1.1